In [ ]:
'''
Trading Algo PScode:
    initialize tradeable stocks list
    Iterate through list of stocks:
        if stock is tradeable
        AND if stock's most recent news is positive
        --IGNORING AND if stock most recent news has been posted in last 30 minutes:
        AND stock volume < avg. volume:
            add stock ticker and number of clicks to tradeable stock list
    --IGNORING FOR NOW: sort tradeable stocks list by number of clicks
    for each stock ticker:
        calculate 20 minute moving average
            stocks_price_day = rs.stocks.get_historicals(inputSymbols='tsla', span='day')
            get last 4 open prices and divide by 4 = 20 minute moving average
        #see if stock is trending upwards
        if 20 minute moving average > current_price:
            #place buy-limit order to buy at the current price 
            robin_stocks.orders.order_buy_limit(symbol, quantity, limitPrice, timeInForce='gtc', extendedHours=False)
'''


''''
Trading Algo PScode 1 helper
    filter for tradeable stocks
    filter for most recent news
    filter for low stock volume
    filter for 20 minute moving average
    sort list by moving average
        send CSV to cloud 
'''

'''
Trading Algo PScode 1
    get CSV from cloud
    check that its a new list of stocks
        the most recent news story should be within 45 minutes of current
        time?
        OR 
        increment the csv file name by 1 so I know its a new version?
    iterate through stocks in CSV file
        place a buy limit order
        confirm stock is purchased
        set sell limit order      
'''

#stock_filter_helper
'''
get_tradeable_stocks
    CSV file = [index, stock tickers]
    filter_recent_news(tradeable_stocks_csv)
filter_recent_news(tradeable_stocks_csv: csv file)
    CSV = [index, stock_tickers, news_story_title]
    remove_high_vol_stocks(filtered_recent_news_csv)
remove_high_vol_stocks(filtered_recent_news_csv: csv file)
    CSV = [index, stock_tickers, news_story_title]
    get_trending_stocks(filtered_high_volume_csv)
get_trending_stocks(filtered_high_volume_csv: csv file):
    CSV = [index, stock_tickers, news_story_title, trendrate]
    purchase_list = sort CSV by trendrate
    push_to_cloud(purchase_list)
push_to_cloud(purchase_list_csv: csv file):
    #can I add a number to the csv file name 
    #increment that value each time?
    
'''

#stock trader
'''
while markets are still open
    get_cloud_csv
        if csv file is old
            wait 1 minute
            check again
        else
            return new csv file
    iterate through stocks
        buy stocks
'''




In [72]:
'''
#filter for stocks I can't afford given buying power
def filter_exp_stocks(all_tradeable_stocks):
    index = 0
    afford_stock_list = []
    while index < len(all_tradeable_stocks):
        stock = all_tradeable_stocks[index]
        if enough_money(stock):
            afford_stock_list.append(stock)
        index+=1
    return afford_stock_list
'''
'''
startTime = datetime.now()
afford_stock_list = filter_exp_stocks(all_tradeable_stocks)
print(datetime.now() - startTime)   
'''

'\nstartTime = datetime.now()\nafford_stock_list = filter_exp_stocks(all_tradeable_stocks)\nprint(datetime.now() - startTime)   \n'

In [ ]:
#one time install
#!pip install --user nbimporter

In [51]:
import kfp
import kfp.components as comp
import kfp.compiler as pipe_comp
import robin_stocks as rs
import pandas as pd
import requests
import numpy as np
import nbimporter
from typing import NamedTuple
from collections import namedtuple
from google.cloud import storage
from datetime import timedelta, datetime
import time
import csv

In [57]:
#include functions from notebooks
%run auth.ipynb
%run recent_stock_news.ipynb
%run check_volume.ipynb
%run get_trending_stocks.ipynb
%run purchase_stock.ipynb
%run tradeable_stocks.ipynb

In [58]:
#making notebook functions available
import auth
import recent_stock_news
import check_volume
import get_trending_stocks
import purchase_stock
import tradeable_stocks

In [96]:
def push_to_cloud(my_trending_stocks):
    file_name = "purchase_list_V"
    a=my_trending_stocks
    with open(file_name + ".csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(a)
    storage_client = storage.Client(project="bright-voltage-248522")
    bucket = storage_client.get_bucket('arys_bucket1')
    blob = bucket.blob("stock_purchase_list")
    blob.upload_from_filename(file_name+".csv")

In [73]:
#runs all the filters to get purchase list
def push_stock_purchase_list_to_cloud():
    all_tradeable_stocks = get_tradeable_stocks()
    recent_stock_news1 = filter_recent_news(all_tradeable_stocks)
    low_vol_stocks = remove_high_vol_stocks(recent_stock_news1)
    my_trending_stocks = filter_trending_stock(low_vol_stocks)
    
    push_to_cloud(my_trending_stocks)

In [125]:
def get_purchase_list_from_cloud():
    storage_client = storage.Client(project="bright-voltage-248522")
    bucket = storage_client.get_bucket('arys_bucket1')
    blob = bucket.blob("stock_purchase_list")
    data = blob.download_to_filename('stock_purchase_list_d.csv')
    data_csv = pd.read_csv('stock_purchase_list_d.csv', parse_dates=True, names=['symbol', 'news_title', 'trendrate'])
    return data_csv

In [4]:
get_header = rh_auth(username='', password='')

In [97]:
current_time = datetime.now()
current_time_form = (current_time.hour*100) + current_time.minute
while current_time_form >= 930 and current_time_form <= 1735:
    #push purchase list to cloud
    print(current_time.hour, current_time.minute)
    push_stock_purchase_list_to_cloud()
    current_time = datetime.now()
    print(current_time.hour, current_time.minute)
    current_time_form = (current_time.hour*100) + current_time.minute

17 9
Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
Loading page 20 ...
Loading page 21 ...
Loading page 22 ...
Loading page 23 ...
Loading page 24 ...
Loading page 25 ...
Loading page 26 ...
Loading page 27 ...
Loading page 28 ...
Loading page 29 ...
Loading page 30 ...
Loading page 31 ...
Loading page 32 ...
Loading page 33 ...
Loading page 34 ...
Loading page 35 ...
Loading page 36 ...
Loading page 37 ...
Loading page 38 ...
Loading page 39 ...
Loading page 40 ...
Loading page 41 ...
Loading page 42 ...
Loading page 43 ...
Loading page 44 ...
Loading page 45 ...
Loading page 46 ...
Loading page 47 ...
Loading page 48 ...
Loading page 49 ...
Loading page 50 ...

/Users/asharifi/anaconda3/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


17 20
17 20
Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
Loading page 20 ...
Loading page 21 ...
Loading page 22 ...
Loading page 23 ...
Loading page 24 ...
Loading page 25 ...
Loading page 26 ...
Loading page 27 ...
Loading page 28 ...
Loading page 29 ...
Loading page 30 ...
Loading page 31 ...
Loading page 32 ...
Loading page 33 ...
Loading page 34 ...
Loading page 35 ...
Loading page 36 ...
Loading page 37 ...
Loading page 38 ...
Loading page 39 ...
Loading page 40 ...
Loading page 41 ...
Loading page 42 ...
Loading page 43 ...
Loading page 44 ...
Loading page 45 ...
Loading page 46 ...
Loading page 47 ...
Loading page 48 ...
Loading page 49 ...
Loading page

/Users/asharifi/anaconda3/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


17 30
17 30
Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
Loading page 20 ...
Loading page 21 ...
Loading page 22 ...
Loading page 23 ...
Loading page 24 ...
Loading page 25 ...
Loading page 26 ...
Loading page 27 ...
Loading page 28 ...
Loading page 29 ...
Loading page 30 ...
Loading page 31 ...
Loading page 32 ...
Loading page 33 ...
Loading page 34 ...
Loading page 35 ...
Loading page 36 ...
Loading page 37 ...
Loading page 38 ...
Loading page 39 ...
Loading page 40 ...
Loading page 41 ...
Loading page 42 ...
Loading page 43 ...
Loading page 44 ...
Loading page 45 ...
Loading page 46 ...
Loading page 47 ...
Loading page 48 ...
Loading page 49 ...
Loading page

/Users/asharifi/anaconda3/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


17 41


In [ ]:
#add file versioning
#download the file from cloud storage
#connect fields to buy_stock(sorted_upward_trending_stocks)

In [ ]:
#WARNING: ACTUALLY PURCHASES STOCK based on your available cash
#will continue until you have no more cash or there are no more trending
#stock to purchase
buy_stock(sorted_upward_trending_stocks)